## Functions

In [1]:
import LyndonWords
import importlib
import AnalysisFuncs
importlib.reload(LyndonWords)
importlib.reload(AnalysisFuncs)
from AnalysisFuncs import *
from LyndonWords import *
import itertools
from functools import partial
import multiprocessing as mp
from tqdm.notebook import tqdm
import math
import numpy as np
def gen_permutations(type,n):
    for perm in itertools.permutations(np.array(range(n+1))):
        yield rootSystem(perm,type)

### Monotonicity Stuff

In [2]:
def monotonicity_conj(rootsys:rootSystem,wordToCheck:word):
    wordPlusDelta = rootsys.get_words(wordToCheck.weights + wordToCheck.delta)[0]
    parsedFormat = rootsys.parse_to_delta_format(wordPlusDelta)
    if(type(parsedFormat[0]) is list):
        return True
    return False
#TODO: Updated bases on new claim on monotonicity
def check_monotone_increasing_imp_cond(rootSystemPermutations):
    rootsys:rootSystem
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.get_words(rootsys.delta)
        temp = rootsys.check_monotonicity("Increasing")
        monotoneIncreasing = [rootsys.get_words(i[0] + rootsys.delta)[0] for i in temp]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(not str(i).startswith(str(j)) and (i.weights - rootsys.delta)[rootsys.ordering[0].rootIndex] > 0):
                    flag = True
                    break
            if(flag):
                continue
            returnArr.append((str(rootsys.ordering),(i.weights - rootsys.delta)))
    return returnArr

In [3]:
def check_cond_imp_monotone_increasing(rootSystemPermutations):
    returnArr = []
    rootsys:rootSystem
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.get_words(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.get_words(base + rootsys.delta)[0]
            if(base[rootsys.ordering[0].rootIndex] == 0):
                continue
            deltaStart = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaHeight] == i)):
                    deltaStart = True
                    break
            if deltaStart:
                continue
            if(rootsys.get_monotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.get_words(base)[0])))
    return returnArr

### Printing and Exporting

In [4]:
def print_by_length(rootSystem:rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.get_words(comb):
                print(" "+str(i))

In [5]:
def print_by_base_word(rootSystem:rootSystem,comb):
    k=0
    is_imaginary = rootSystem.is_imaginary(comb)
    for i in rootSystem.get_affine_words(comb):
        if(is_imaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not is_imaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [6]:
def export_by_length(rootsys:rootSystem,quoteMarks=False):
    file = open("out.txt",'w')
    file.write(str(rootsys.ordering)+"\n")
    for k in range(rootsys.k + 1):
        for i in rootsys.baseWeights:
            comb = i + rootsys.delta * k
            for i in rootsys.get_words(comb):
                if(quoteMarks):
                    file.write(f"'{str(i)}',")
                else:
                    file.write(str(i) + '\n')
    file.close()

### Delta Pattern Detection

In [7]:
def write_delta_types_to_file(rootSystems,file="out.txt",total=0,notOnlyAllLast=True):
    fileobj = open(file,'w')
    fileobj.write("imaginary index, bracketing, splitting type, inserted imaginary index, left factor, right factor\n")
    deltaTypesCol:deltaTypesCollection
    first:rootSystem
    if(total == 0):
        first = next(rootSystems)
        total = math.factorial(first.n+1)-1
        deltaTypesCol = generate_delta_types(first)
        if(notOnlyAllLast):
            if(deltaTypesCol.not_all_last()):
                fileobj.write(deltaTypesCol.to_csv())
    with mp.Pool() as p:
        for deltaTypesCol in tqdm(p.imap_unordered(generate_delta_types,rootSystems),total=total):
            if(notOnlyAllLast):
                if(not deltaTypesCol.not_all_last()):
                    continue
            fileobj.write(deltaTypesCol.to_csv())
    fileobj.close()

In [ ]:
def check_standard_fac_same_parrallel(rootSystems,total=0,k=2):
    results = []
    if(total == 0):
        first = next(rootSystems)
        total = math.factorial(first.n+1)-1
        result = check_standard_fac_same(first)
        if(result != True):
            yield result
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(check_standard_fac_same,k=k),rootSystems,chunksize=5),total=total):
            if(result != True):
                yield result
    return results

### SL Word Algorithm

In [9]:
def check_SL_word_algo_perms(rootsyss,k,equal=False):
    for rootsys in rootsyss:
        for exc in check_SL_word_algo(rootsys,k,equal):
            yield (str(rootsys.ordering),exc,rootsys.parse_to_delta_format(rootsys.get_words(exc)[0]),rootsys.parse_to_delta_format(rootsys.SL_word_algo(exc)))
def check_SL_word_algo(rootsys:rootSystem,k,equal=False):
    for kprime in range(k):
        for baseWeight in rootsys.baseWeights[:-1]:
            weight = baseWeight + kprime*rootsys.delta
            if(equal):
                if(word.letter_list_cmp(rootsys.get_words(weight)[0].string,rootsys.SL_word_algo(weight).string) != 0):
                    yield weight
            else:
                if(word.letter_list_cmp(rootsys.get_words(weight)[0].string,rootsys.SL_word_algo(weight).string) > 0):
                    yield weight 

### Imaginary Words Stuff

In [10]:
def check_h_deltas_repeat(perms):
    excepts = []
    i:rootSystem
    for i in perms:
        hs = [j.hs for j in i.get_affine_words(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

In [11]:
def check_one_smallest_last(perms):
    rootsys:rootSystem
    for rootsys in perms:
        if(rootsys.delta[rootsys.ordering[0].rootIndex] == 1):
            if(rootsys.get_words(rootsys.delta)[0][-1].rootIndex != rootsys.ordering[1].rootIndex):
                yield (str(rootsys.ordering),rootsys.get_words(rootsys.delta)[0].noCommas(),rootsys.ordering[1].rootIndex)

### Rootsystem equivalences

In [12]:
def compare_rootsystems(r1,r2):
    words = list(r1.weightToWordDictionary.values())
    for i in words:
        for j in range(len(i)):
            if(str(i[j]) != str(r2.get_words(i[j].weights)[j])):
                yield (i[j].weights,str(i[j]),str(r2.get_words(i[j].weights)[j]))

In [13]:
def generate_equivalences(permutations):
    equivalences = np.arange(len(permutations))
    for i in range(len(permutations)):
        for j in range(0,i):
            if(equivalences[j] != j):
                continue
            flag = False
            for k in compare_rootsystems(permutations[i],permutations[j]):
                flag =True
                break
            if(not flag):
                yield (permutations[i].ordering,permutations[j].ordering)

### Periodicity

In [14]:
def get_max_periodicity(rootSystems,total=0):
    max = 0
    maxOrdering = ""
    maxRoot = []
    rootsys:rootSystem
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(max_periodicity_rootSystem,rootSystems),total=total):
            if(result.periodicity > max):
                max = result.periodicity
                maxOrdering = result.ordering
                maxRoot= result.maxRoot
    return (max,maxOrdering,maxRoot)

In [15]:
def periodicity_conj(rootsys:rootSystem,deltas, weights) -> int:
    if(weights[rootsys.ordering[0].rootIndex] != 0):
        return 1
    for d in deltas:
        hs = rootsys.cartan_matrix @ d.hs
        f = False   
        for v in range(len(weights)):
            if(v == 0 and weights[v] > 0):
                if(np.dot(rootsys.delta[1:],hs) != 0):
                    f=True
                    break
            else:
                if(weights[v] > 0 and hs[v-1] != 0):
                    f=True
                    break
        if(f):
            break
    s = 0
    for v in range(len(weights)):
            if(v == 0 and weights[v] > 0):
                if(np.dot(rootsys.delta[1:],hs) != 0):
                    s+=1
            else:
                if(weights[v] > 0 and hs[v-1] != 0):
                    s+=weights[v]
    return s

In [16]:
def periodicity_conj_root(rootsys:rootSystem, weights) :
    return periodicity_conj(rootsys,rootsys.get_words(rootsys.delta),weights)

In [17]:
def check_periodicity_conj_rootsystem(rootsys:rootSystem):
    if(rootsys.delta[rootsys.ordering[0].rootIndex] == 1):
        for l in rootsys.baseWeights[:-1]:
            if(l[rootsys.ordering[0].rootIndex] == 0):
                period = rootsys.get_periodicity(l)
                conjperiod = periodicity_conj_root(rootsys,l)
                if(period != conjperiod):
                    yield (str(rootsys.ordering),l,period,conjperiod)

In [18]:
def check_periodicity_conj_perms(perms):
    for p in perms:
        yield check_periodicity_conj_rootsystem(p)

In [19]:
B4 = rootSystem([1,3,4,0,2],'B')
B4.generate_up_to_delta(4)

## Type A

In [136]:
Aperms = list(gen_permutations('A',5))

In [137]:
periodicity_conj_root(rootSystem([0,1,2,3,4],'A'),[0,0,1,0,0])

1

In [138]:
for i in check_periodicity_conj_perms(Aperms):
    print(i)

<generator object check_periodicity_conj_rootsystem at 0x0000027390361990>
<generator object check_periodicity_conj_rootsystem at 0x0000027390363E20>
<generator object check_periodicity_conj_rootsystem at 0x0000027390361990>
<generator object check_periodicity_conj_rootsystem at 0x0000027390363E20>
<generator object check_periodicity_conj_rootsystem at 0x0000027390361990>
<generator object check_periodicity_conj_rootsystem at 0x0000027390363E20>
<generator object check_periodicity_conj_rootsystem at 0x0000027390361990>
<generator object check_periodicity_conj_rootsystem at 0x0000027390363E20>
<generator object check_periodicity_conj_rootsystem at 0x0000027390361990>
<generator object check_periodicity_conj_rootsystem at 0x0000027390363E20>
<generator object check_periodicity_conj_rootsystem at 0x0000027390361990>
<generator object check_periodicity_conj_rootsystem at 0x0000027390363E20>
<generator object check_periodicity_conj_rootsystem at 0x0000027390361990>
<generator object check_p

In [ ]:
for i in check_SL_word_algo_perms(Aperms,3,True):
    print(i)

In [ ]:
for i in check_delta_type_prediction_perms(Aperms):
    print(i)

In [ ]:
write_delta_types_to_file(Aperms,"ADeltaTypes.csv")

In [ ]:
get_max_periodicity(Aperms)

0it [00:00, ?it/s]

KeyboardInterrupt: 

## Type C

In [ ]:
Cperms = gen_permutations('C',5)

In [ ]:
for i in check_periodicity_conj_perms(Cperms):
    print(i)

In [ ]:
for i in check_SL_word_algo_perms(Cperms,3,True):
    print(i)

In [ ]:
for i in check_delta_type_prediction_perms(Cperms):
    print(i)

In [ ]:
write_delta_types_to_file(Cperms,"CDeltaTypes.csv")

  0%|          | 0/719 [00:00<?, ?it/s]

In [ ]:
print(*[f"{str(i[0])}: {i[1]}\n" for i in check_monotone_increasing_imp_cond(Cperms)])

5<4<3<2<1<0: [0 0 0 0 0 1]
 5<4<3<2<1<0: [1 0 0 0 0 1]
 5<4<3<2<1<0: [1 1 0 0 0 1]
 5<4<3<2<1<0: [2 0 0 0 0 1]
 5<4<3<2<1<0: [2 1 0 0 0 1]
 5<4<3<2<1<0: [2 2 0 0 0 1]



In [ ]:
check_cond_imp_monotone_increasing(Cperms)

In [ ]:
check_h_deltas_repeat(Cperms)

[]

In [ ]:
for i in generate_equivalences(Cperms):
    print(f"{i[0]} = {i[1]}")

In [ ]:
get_max_periodicity(Cperms)

(5, '0<3<1<4<2', array([0, 0, 2, 2, 1]))

## Type G

In [165]:
G2perms = gen_permutations('G',2)

In [166]:
check_standard_fac_same_parrallel(G2perms,k=3)

  0%|          | 0/5 [00:00<?, ?it/s]

[]

In [ ]:
for i in check_periodicity_conj_perms(G2perms):
    print(i)

In [ ]:
get_max_periodicity(G2perms)

(5, '1<0<2', array([1, 1, 3]))

In [ ]:
for i in check_SL_word_algo_perms(G2perms,5,equal=True):
    print(i)

('2<0<1', array([1, 3, 4]), [[2, 1], '21'], [[1, 1]])
('2<0<1', array([2, 3, 5]), [[2, 1], '2210'], ['2210', [1, 1]])
('2<0<1', array([2, 5, 7]), [[2, 2], '21'], [[1, 1]])
('2<0<1', array([3, 5, 8]), [[2, 2], '2210'], ['2210', [1, 2]])
('2<0<1', array([ 3,  7, 10]), [[2, 3], '21'], [[1, 1]])
('2<0<1', array([ 4,  7, 11]), [[2, 3], '2210'], ['2210', [1, 3]])
('2<0<1', array([ 4,  9, 13]), [[2, 4], '21'], [[1, 1]])
('2<0<1', array([ 5,  9, 14]), [[2, 4], '2210'], ['2210', [1, 4]])
('2<1<0', array([1, 3, 4]), [[2, 1], '21'], [[1, 1]])
('2<1<0', array([2, 3, 5]), [[2, 1], '2210'], ['2210', [1, 1]])
('2<1<0', array([2, 5, 7]), [[2, 2], '21'], [[1, 1]])
('2<1<0', array([3, 5, 8]), [[2, 2], '2210'], ['2210', [1, 2]])
('2<1<0', array([ 3,  7, 10]), [[2, 3], '21'], [[1, 1]])
('2<1<0', array([ 4,  7, 11]), [[2, 3], '2210'], ['2210', [1, 3]])
('2<1<0', array([ 4,  9, 13]), [[2, 4], '21'], [[1, 1]])
('2<1<0', array([ 5,  9, 14]), [[2, 4], '2210'], ['2210', [1, 4]])


In [ ]:
for i in check_delta_type_prediction_perms(G2perms):
    print(i)

In [ ]:
write_delta_types_to_file(G2perms,"GDeltaTypes.csv",False)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
check_cond_imp_monotone_increasing(G2perms)

[('1<0<2', '1,2,2,2'), ('1<0<2', '1,2,2,2,0')]

In [ ]:
check_monotone_increasing_imp_cond(G2perms)

[]

In [ ]:
for i in G2perms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(G2perms)

[]

In [ ]:
for i in generate_equivalences(G2perms):
    print(f"{i[0]} = {i[1]}")

## Type B

In [ ]:
Bperms = gen_permutations('B',5)

In [ ]:
for i in check_SL_word_algo_perms(Bperms,4):
    print(i)

In [ ]:
for i in check_SL_word_algo_perms(Bperms):
    print(i)

TypeError: check_SL_word_algo_perms() missing 1 required positional argument: 'k'

In [ ]:
for i in check_delta_type_prediction_perms(Bperms):
    print(i)

In [ ]:
write_delta_types_to_file(Bperms,"BDeltaTypes.csv")

  0%|          | 0/719 [00:00<?, ?it/s]

In [ ]:
check_cond_imp_monotone_increasing(Bperms)

In [ ]:
check_monotone_increasing_imp_cond(Bperms)

[]

In [ ]:
for i in Bperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(Bperms)

[]

In [ ]:
for i in generate_equivalences(Bperms):
    print(f"{i[0]} = {i[1]}")

## Type D

In [ ]:
Dperms = gen_permutations('D',6)

In [ ]:
for i in check_SL_word_algo_perms(Dperms,3):
    print(i)

In [ ]:
for i in check_delta_type_prediction_perms(Dperms):
    print(i)

In [ ]:
write_delta_types_to_file(Dperms,"DDeltaTypes.csv")

  0%|          | 0/5039 [00:00<?, ?it/s]

In [ ]:
check_monotone_increasing_imp_cond(Dperms)

[('5<4<3<2<1<0', array([0, 0, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([0, 1, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([1, 1, 0, 0, 0, 1]))]

In [ ]:
check_cond_imp_monotone_increasing(Dperms)

[('2<0<4<5<1<3', '2,3,1,5,4,3'),
 ('2<0<4<5<1<3', '2,3,1,5,4,3,0'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3,0'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3,1'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3,1'),
 ('2<4<0<5<1<3', '2,3,1,5,4,3'),
 ('2<4<0<5<1<3', '2,3,1,5,0,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,1,4,3'),
 ('2<4<5<0<1<3', '2,3,1,5,4,3'),
 ('2<4<5<0<1<3', '2,3,1,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,1,5,4,3'),
 ('2<5<0<4<1<3', '2,3,1,4,5,3'),
 ('2<5<0<4<1<3', '2,3,1,4,0,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,1,5,3'),
 ('2<5<4<0<1<3', '2,3,1,4,5,3'),
 ('2<5<4<0<1<3', '2,3,1,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,1,4,5,3'),
 ('3<0<1<4<5<2', '3,2,5,1,0,2'),
 ('3<0<1<4<5<2', '3,2,5,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,5,1,0,2'),
 ('3<0<4<1<5<2', '3,2,5,1,0,2'),
 ('3<0<4<1<5<2'

In [ ]:
for i in Dperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(Dperms)

[]

In [ ]:
for i in generate_equivalences(Dperms):
    print(f"{i[0]} = {i[1]}")

## Type E

In [21]:
Eperms = gen_permutations('E',7)

In [22]:
for i in check_standard_fac_same_parrallel(Eperms,k=3):
    print(i)

  0%|          | 0/40319 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
write_delta_types_to_file(Eperms,'E7DeltaTypes.csv')

  0%|          | 0/40319 [00:00<?, ?it/s]

In [ ]:
for i in check_periodicity_conj_perms(Eperms):
    print(i)

In [ ]:
for i in check_one_smallest_last(Eperms):
    print(i)

In [ ]:
for i in check_SL_word_algo_perms(Eperms,3):
    print(i)

KeyboardInterrupt: 

In [ ]:
for i in check_delta_type_prediction_perms(Eperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
check_h_deltas_repeat(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

[]

In [ ]:
for i in Eperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
write_delta_types_to_file(Eperms,"E6DeltaTypes.csv")

  0%|          | 0/5040 [00:00<?, ?it/s]

In [ ]:
for i in generate_equivalences(Eperms):
    print(f"{i[0]} = {i[1]}")

0<1<2<4<5<6<3 = 0<1<2<4<5<3<6
0<1<2<5<4<6<3 = 0<1<2<5<4<3<6
0<1<3<4<5<6<2 = 0<1<3<4<5<2<6
0<1<3<5<4<6<2 = 0<1<3<5<4<2<6
0<1<4<2<5<6<3 = 0<1<4<2<5<3<6
0<1<4<3<5<6<2 = 0<1<4<3<5<2<6
0<1<4<5<2<6<3 = 0<1<4<5<2<3<6
0<1<4<5<3<6<2 = 0<1<4<5<3<2<6
0<1<4<5<6<2<3 = 0<1<4<5<2<3<6
0<1<4<5<6<2<3 = 0<1<4<5<2<6<3
0<1<4<5<6<3<2 = 0<1<4<5<3<2<6
0<1<4<5<6<3<2 = 0<1<4<5<3<6<2
0<1<5<2<4<6<3 = 0<1<5<2<4<3<6
0<1<5<3<4<6<2 = 0<1<5<3<4<2<6
0<1<5<4<2<6<3 = 0<1<5<4<2<3<6
0<1<5<4<3<6<2 = 0<1<5<4<3<2<6
0<1<5<4<6<2<3 = 0<1<5<4<2<3<6
0<1<5<4<6<2<3 = 0<1<5<4<2<6<3
0<1<5<4<6<3<2 = 0<1<5<4<3<2<6
0<1<5<4<6<3<2 = 0<1<5<4<3<6<2
0<2<1<4<5<6<3 = 0<2<1<4<5<3<6
0<2<1<5<4<6<3 = 0<2<1<5<4<3<6
0<2<3<4<5<1<6 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<5<1<6
0<2<3<4<6<1<5 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<6<1<5
0<2<3<5<4<6<1 = 0<2<3<5<4<1<6
0<2<3<5<6<1<4 = 0<2<3<5<1<6<4
0<2<3<6<4<5<1 = 0<2<3<6<4<1<5
0<2<4<1<5<6<3 = 0<2<4<1<5<3<6
0<2<4<3<5<1<6 = 0<2<4<3<1<5<6
0<2<4<3<5<

KeyboardInterrupt: 

In [ ]:
get_max_periodicity(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

(10, '2<0<1<4<6<5<3', array([1, 0, 1, 1, 1, 1, 0]))

## Type F

In [167]:
Fperms = gen_permutations('F',4)

In [168]:
check_standard_fac_same_parrallel(Fperms,k=3)

  0%|          | 0/119 [00:00<?, ?it/s]

[]

In [ ]:
for i in check_periodicity_conj_perms(Fperms):
    print(i)

In [ ]:
for i in check_SL_word_algo_perms(Fperms,3,True):
    print(i)

In [ ]:
for i in check_delta_type_prediction_perms(Fperms):
    print(i)

('F', '1<0<2<3<4', 1, 'cofac', 'k3start')
('F', '1<0<2<4<3', 1, 'cofac', 'k3start')
('F', '1<0<3<2<4', 1, 'cofac', 'k3start')
('F', '1<0<3<4<2', 1, 'cofac', 'k3start')
('F', '1<0<4<2<3', 1, 'cofac', 'k3start')
('F', '1<0<4<3<2', 1, 'cofac', 'k3start')
('F', '1<2<0<3<4', 1, 'cofac', 'k3start')
('F', '1<2<0<4<3', 1, 'cofac', 'k3start')
('F', '1<2<3<0<4', 1, 'cofac', 'k3start')
('F', '1<2<3<4<0', 1, 'cofac', 'k3start')
('F', '1<2<4<0<3', 1, 'cofac', 'k3start')
('F', '1<2<4<3<0', 1, 'cofac', 'k3start')
('F', '1<3<0<2<4', 1, 'cofac', 'k3start')
('F', '1<3<0<2<4', 3, 'k3start', 'k3start')
('F', '1<3<0<4<2', 1, 'cofac', 'k3start')
('F', '1<3<0<4<2', 4, 'k3start', 'k3start')
('F', '1<3<2<0<4', 1, 'cofac', 'k3start')
('F', '1<3<2<4<0', 1, 'cofac', 'k3start')
('F', '1<3<4<0<2', 1, 'cofac', 'k3start')
('F', '1<3<4<0<2', 4, 'k3start', 'k3start')
('F', '1<3<4<2<0', 1, 'cofac', 'k3start')
('F', '1<4<0<2<3', 1, 'cofac', 'k3start')
('F', '1<4<0<3<2', 1, 'cofac', 'k3start')
('F', '1<4<2<0<3', 1, 'cofac

IndexError: index 4 is out of bounds for axis 0 with size 4

In [ ]:
write_delta_types_to_file(Fperms,"FDeltaTypes.csv",notOnlyAllLast=False)

  0%|          | 0/119 [00:00<?, ?it/s]

In [ ]:
check_monotone_increasing_imp_cond(Fperms)

UnboundLocalError: cannot access local variable 'returnArr' where it is not associated with a value

In [ ]:
check_cond_imp_monotone_increasing(Fperms)

In [ ]:
for i in Fperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
check_h_deltas_repeat(Fperms)

[]

In [ ]:
get_max_periodicity(Fperms,math.factorial(5))

  0%|          | 0/120 [00:00<?, ?it/s]

(9, '0<2<3<1<4', array([0, 2, 3, 4, 2]))

In [ ]:
E6 = rootSystem([0,4,3,2,1,6,5,7],'E')
E6.generate_up_to_delta(25)
print(get_max_periodicity([E6]))

(12, '0<4<3<2<1<6<5<7', array([0, 1, 2, 3, 4, 3, 1, 2]))


In [ ]:
for i in generate_equivalences(Fperms):
    print(f"{i[0]} = {i[1]}")